In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.master("local[10]").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
spark._jvm.decoders.HRTDCDecoder.registerUDF(spark._jsparkSession)
spark._jvm.decoders.HRTDCUnpairedDecoder.registerUDF(spark._jsparkSession)

df = spark.read.parquet("/home/h487/data/parquet/run0020.parquet")

# Filter SRPPAC strip data and decode
df_src = df.filter("femType==7 and femId==615").select("data").withColumn("decoded",F.expr("decode_hrtdc_unpaired_segdata(data)"))
df_src = df_src.select("decoded.*").select("hbf.*","data").select("hbfNumber","data").filter("array_size(decoded.data)>0")
df_src = df_src.withColumn("ex",F.explode("data")).select("hbfNumber","ex.*").withColumnRenamed("tot","edge")
df_src.filter("hbfNumber=10057285 and ch=41").show()

# Filter SRPPAC anode data and decode
df_sra = df.filter("femType==5 and femId==614").select("data").withColumn("decoded",F.expr("decode_hrtdc_segdata(data)"))
df_sra = df_sra.select("decoded.*").select("hbf.*","data").select("hbfNumber","data").filter("array_size(decoded.data)>0")
df_sra = df_sra.withColumn("ex",F.explode("data")).select("hbfNumber","ex.*").filter("ch==0")
df_sra.show(5)

+---------+---+---------+----+
|hbfNumber| ch|     time|edge|
+---------+---+---------+----+
| 10057285| 41|108931813|   1|
| 10057285| 41|109027119|   0|
+---------+---+---------+----+

+---------+---+---------+------+
|hbfNumber| ch|     time|   tot|
+---------+---+---------+------+
| 10057285|  0|108619151|153540|
| 10057473|  0|226489218|153508|
| 10057661|  0|333310572|153522|
| 10057849|  0|451976248|153506|
| 10058038|  0| 12587130|153479|
+---------+---+---------+------+
only showing top 5 rows
